This notebook demonstrates the API of the `cvat_sdk.pytorch` module in CVAT SDK.
It is part of the supplementary materials for the article "PyTorch adapter in the CVAT SDK".

To run it, first fill in your CVAT access credentials in the cell below.
You also need to run the `upload-flowers.py` script and fill in the ID of the training task that it has printed.

In [ ]:
CVAT_HOST = 'app.cvat.ai' # the hostname of your CVAT instance
CVAT_USER = '...' # your username
CVAT_PASS = '...' # your password

TRAIN_TASK_ID = ...

First, we need to create a CVAT API client.
This will log into the CVAT server and verify the credentials.

In [ ]:
import logging, os
from cvat_sdk import *

# configure logging to see what the SDK is doing behind the scenes
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
client = make_client(CVAT_HOST, credentials=(CVAT_USER, CVAT_PASS))

Now we can create a `TaskVisionDataset` instance representing our training task.

In [ ]:
from cvat_sdk.pytorch import *

train_set = TaskVisionDataset(client, TRAIN_TASK_ID)

We can verify that `train_set` is an instance of the PyTorch `Dataset` class:

In [ ]:
import torch.utils.data
isinstance(train_set, torch.utils.data.Dataset)

As such, we can query its length:

In [ ]:
len(train_set)

And we can query individual samples. The first component of each sample is an image (an instance of the `PIL.Image` class):

In [ ]:
train_set[0][0]

And the second component is a `Target` object containing the annotations associated with the image:

In [ ]:
train_set[0][1]

To simplify the target component in a simple classification scenario,
you can use the `ExtractSingleLabelIndex` transform.

In [ ]:
train_set = TaskVisionDataset(client, TRAIN_TASK_ID, 
      target_transform=ExtractSingleLabelIndex())

When this transform is applied,
the target component becomes a zero-dimensional tensor containing the label index.

In [ ]:
for i in range(3):
    print(train_set[i])

To use the samples as inputs to PyTorch models,
you'll also need to transform the image component.
torchvision already includes a variety of image transforms,
which you can use via the `transform` argument.
For example:

In [ ]:
import torchvision.transforms as transforms

train_set = TaskVisionDataset(client, TRAIN_TASK_ID,
    transform=transforms.ToTensor(),
    target_transform=ExtractSingleLabelIndex())

Now the image component is a PyTorch tensor too.

In [ ]:
train_set[0]